### Konstantina Andronikou

## Transitive Vs Intransitive Verbs 

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Intransitive 

In [4]:
# remove warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Carla pays")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'pays',
   'description': '[ARG1: Carla] [V: pays]',
   'tags': ['B-ARG1', 'B-V']}],
 'words': ['Carla', 'pays']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Carla pays"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'pays',
     'description': '[ARG1: Carla] [V: pays]',
     'tags': ['B-ARG1', 'B-V']}],
   'words': ['Carla', 'pays']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays', meta=True, nsamples=1000, remove_duplicates = True)

for k, v in t.items():
    print(k,v)

meta [{'first_name': 'Anne'}, {'first_name': 'Elaine'}, {'first_name': 'Jonathan'}, {'first_name': 'Albert'}, {'first_name': 'Benjamin'}, {'first_name': 'Charlotte'}, {'first_name': 'Roger'}, {'first_name': 'Jason'}, {'first_name': 'Arthur'}, {'first_name': 'Carolyn'}, {'first_name': 'Brian'}, {'first_name': 'Angela'}, {'first_name': 'Albert'}, {'first_name': 'Samuel'}, {'first_name': 'Amy'}, {'first_name': 'Edward'}, {'first_name': 'Charlotte'}, {'first_name': 'Walter'}, {'first_name': 'Virginia'}, {'first_name': 'Ben'}, {'first_name': 'Michael'}, {'first_name': 'Katherine'}, {'first_name': 'Walter'}, {'first_name': 'Sandra'}, {'first_name': 'Melissa'}, {'first_name': 'Janet'}, {'first_name': 'Bruce'}, {'first_name': 'Jessica'}, {'first_name': 'Heather'}, {'first_name': 'Nicole'}, {'first_name': 'Carol'}, {'first_name': 'Pamela'}, {'first_name': 'Alan'}, {'first_name': 'Wendy'}, {'first_name': 'Andrew'}, {'first_name': 'Janet'}, {'first_name': 'Anne'}, {'first_name': 'Julie'}, {'first

In [12]:
with open('Dataset_BERT/Intransitive.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
test = MFT(**t, name = 'detect_arg0name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Intransitive_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    287 (28.7%)

Example fails:
[ARG1: Harold] [V: pays]
----
[ARG1: Grace] [V: pays]
----
[ARG1: Ann] [V: pays]
----
['Anne', 'pays'] [ True]
['Elaine', 'pays'] [ True]
['Jonathan', 'pays'] [ True]
['Albert', 'pays'] [ True]
['Benjamin', 'pays'] [ True]
['Charlotte', 'pays'] [ True]
['Roger', 'pays'] [ True]
['Jason', 'pays'] [ True]
['Arthur', 'pays'] [ True]
['Carolyn', 'pays'] [ True]
['Brian', 'pays'] [ True]
['Angela', 'pays'] [False]
['Albert', 'pays'] [ True]
['Samuel', 'pays'] [ True]
['Amy', 'pays'] [ True]
['Edward', 'pays'] [ True]
['Charlotte', 'pays'] [ True]
['Walter', 'pays'] [ True]
['Virginia', 'pays'] [False]
['Ben', 'pays'] [ True]
['Michael', 'pays'] [ True]
['Katherine', 'pays'] [ True]
['Walter', 'pays'] [ True]
['Sandra', 'pays'] [False]
['Melissa', 'pays'] [ True]
['Janet', 'pays'] [ True]
['Bruce', 'pays'] [False]
['Jessica', 'pays'] [ True]
['Heather', 'pays'] [ True]
['Nicole', 'pays'] [ True]
['Car

In [15]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Intransitive_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive 

In [16]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [17]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [18]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [19]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [20]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Alex'}, {'first_name': 'Bill'}, {'first_name': 'Ben'}, {'first_name': 'Frederick'}, {'first_name': 'John'}, {'first_name': 'Michael'}, {'first_name': 'Patrick'}, {'first_name': 'Jonathan'}, {'first_name': 'Suzanne'}, {'first_name': 'Rose'}, {'first_name': 'Don'}, {'first_name': 'Lucy'}, {'first_name': 'Greg'}, {'first_name': 'Ralph'}, {'first_name': 'Michelle'}, {'first_name': 'Evelyn'}, {'first_name': 'Mike'}, {'first_name': 'Ryan'}, {'first_name': 'Charlotte'}, {'first_name': 'Eric'}, {'first_name': 'Louise'}, {'first_name': 'Leslie'}, {'first_name': 'Kelly'}, {'first_name': 'Sue'}, {'first_name': 'Albert'}, {'first_name': 'Steve'}, {'first_name': 'Harriet'}, {'first_name': 'Johnny'}, {'first_name': 'Alfred'}, {'first_name': 'Nancy'}, {'first_name': 'Sophie'}, {'first_name': 'Billy'}, {'first_name': 'Ian'}, {'first_name': 'Peter'}, {'first_name': 'Adam'}, {'first_name': 'Gary'}, {'first_name': 'Roy'}, {'first_name': 'George'}, {'first_name': 'Ralph'}, {'first_na

In [21]:
with open('Dataset_BERT/Transitive.txt', 'w') as f:
    print(t.data, file = f)

In [22]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Transitive_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Alex', 'pays', 'for', 'coffee', '.'] [ True]
['Bill', 'pays', 'for', 'coffee', '.'] [ True]
['Ben', 'pays', 'for', 'coffee', '.'] [ True]
['Frederick', 'pays', 'for', 'coffee', '.'] [ True]
['John', 'pays', 'for', 'coffee', '.'] [ True]
['Michael', 'pays', 'for', 'coffee', '.'] [ True]
['Patrick', 'pays', 'for', 'coffee', '.'] [ True]
['Jonathan', 'pays', 'for', 'coffee', '.'] [ True]
['Suzanne', 'pays', 'for', 'coffee', '.'] [ True]
['Rose', 'pays', 'for', 'coffee', '.'] [ True]
['Don', 'pays', 'for', 'coffee', '.'] [ True]
['Lucy', 'pays', 'for', 'coffee', '.'] [ True]
['Greg', 'pays', 'for', 'coffee', '.'] [ True]
['Ralph', 'pays', 'for', 'coffee', '.'] [ True]
['Michelle', 'pays', 'for', 'coffee', '.'] [ True]
['Evelyn', 'pays', 'for', 'coffee', '.'] [ True]
['Mike', 'pays', 'for', 'coffee', '.'] [ True]
['Ryan', 'pays', 'for', 'coffee', '.'] [ True]
['Charlotte', 'pays', 'for', 'coffee', '.'] [ True]
['Eric

['Barbara', 'pays', 'for', 'coffee', '.'] [ True]
['Thomas', 'pays', 'for', 'coffee', '.'] [ True]
['Diane', 'pays', 'for', 'coffee', '.'] [ True]
['Stephanie', 'pays', 'for', 'coffee', '.'] [ True]
['Frederick', 'pays', 'for', 'coffee', '.'] [ True]
['Charlotte', 'pays', 'for', 'coffee', '.'] [ True]
['Samuel', 'pays', 'for', 'coffee', '.'] [ True]
['Donna', 'pays', 'for', 'coffee', '.'] [ True]
['Virginia', 'pays', 'for', 'coffee', '.'] [ True]
['Christine', 'pays', 'for', 'coffee', '.'] [ True]
['Brian', 'pays', 'for', 'coffee', '.'] [ True]
['Dave', 'pays', 'for', 'coffee', '.'] [ True]
['Amy', 'pays', 'for', 'coffee', '.'] [ True]
['Bob', 'pays', 'for', 'coffee', '.'] [ True]
['Ken', 'pays', 'for', 'coffee', '.'] [ True]
['Edward', 'pays', 'for', 'coffee', '.'] [ True]
['Chris', 'pays', 'for', 'coffee', '.'] [ True]
['Marie', 'pays', 'for', 'coffee', '.'] [ True]
['Johnny', 'pays', 'for', 'coffee', '.'] [ True]
['Joe', 'pays', 'for', 'coffee', '.'] [ True]
['Patricia', 'pays', 'fo

In [23]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Transitive_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive-Intransitive Robustness Test

In [24]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Jonathan'}, {'first_name': 'Suzanne'}, {'first_name': 'Bill'}, {'first_name': 'Hugh'}, {'first_name': 'Sandra'}, {'first_name': 'Carol'}, {'first_name': 'Katie'}, {'first_name': 'John'}, {'first_name': 'Suzanne'}, {'first_name': 'Benjamin'}, {'first_name': 'Hugh'}, {'first_name': 'Kevin'}, {'first_name': 'Ruth'}, {'first_name': 'David'}, {'first_name': 'Patrick'}, {'first_name': 'John'}, {'first_name': 'Evelyn'}, {'first_name': 'Tony'}, {'first_name': 'Linda'}, {'first_name': 'Jean'}, {'first_name': 'Thomas'}, {'first_name': 'Heather'}, {'first_name': 'Alex'}, {'first_name': 'Christine'}, {'first_name': 'Alison'}, {'first_name': 'Florence'}, {'first_name': 'Sarah'}, {'first_name': 'Charlie'}, {'first_name': 'Bobby'}, {'first_name': 'Lynn'}, {'first_name': 'Jim'}, {'first_name': 'Elaine'}, {'first_name': 'Lynn'}, {'first_name': 'Charlie'}, {'first_name': 'Alexander'}, {'first_name': 'Edwin'}, {'first_name': 'Martha'}, {'first_name': 'Kate'}, {'first_name': 'Amanda'

In [25]:
with open('Dataset_BERT/Intransitive_Rob_BERT.txt', 'w') as f:
    print(t.data, file = f)

In [26]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Intransitive_Rob_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    30 (3.0%)

Example fails:
[ARG1: Diana] [V: payed] .
----
[ARG1: Bill] [V: payed] .
----
[ARG1: Diana] [V: payed] .
----
['Jonathan', 'payed', '.'] [ True]
['Suzanne', 'payed', '.'] [ True]
['Bill', 'payed', '.'] [False]
['Hugh', 'payed', '.'] [ True]
['Sandra', 'payed', '.'] [ True]
['Carol', 'payed', '.'] [ True]
['Katie', 'payed', '.'] [ True]
['John', 'payed', '.'] [ True]
['Suzanne', 'payed', '.'] [ True]
['Benjamin', 'payed', '.'] [ True]
['Hugh', 'payed', '.'] [ True]
['Kevin', 'payed', '.'] [ True]
['Ruth', 'payed', '.'] [ True]
['David', 'payed', '.'] [ True]
['Patrick', 'payed', '.'] [ True]
['John', 'payed', '.'] [ True]
['Evelyn', 'payed', '.'] [ True]
['Tony', 'payed', '.'] [ True]
['Linda', 'payed', '.'] [ True]
['Jean', 'payed', '.'] [ True]
['Thomas', 'payed', '.'] [ True]
['Heather', 'payed', '.'] [ True]
['Alex', 'payed', '.'] [ True]
['Christine', 'payed', '.'] [ True]
['Alison', 'payed', '.'] [ True]
['

['Kathryn', 'payed', '.'] [ True]
['Daniel', 'payed', '.'] [ True]
['Alex', 'payed', '.'] [ True]
['Eric', 'payed', '.'] [ True]
['Anna', 'payed', '.'] [ True]
['Henry', 'payed', '.'] [ True]
['Nicole', 'payed', '.'] [ True]
['Sandra', 'payed', '.'] [ True]
['Kim', 'payed', '.'] [ True]
['Jack', 'payed', '.'] [ True]
['Lucy', 'payed', '.'] [ True]
['Dan', 'payed', '.'] [ True]
['Suzanne', 'payed', '.'] [ True]
['Steve', 'payed', '.'] [ True]
['Ken', 'payed', '.'] [ True]
['Don', 'payed', '.'] [ True]
['Susan', 'payed', '.'] [ True]
['Suzanne', 'payed', '.'] [ True]
['Leslie', 'payed', '.'] [ True]
['Stephanie', 'payed', '.'] [ True]
['Carol', 'payed', '.'] [ True]
['Louis', 'payed', '.'] [ True]
['Stephen', 'payed', '.'] [ True]
['Francis', 'payed', '.'] [ True]
['Sophie', 'payed', '.'] [ True]
['Charlie', 'payed', '.'] [ True]
['Marilyn', 'payed', '.'] [ True]
['Ray', 'payed', '.'] [ True]
['Matt', 'payed', '.'] [ True]
['Alex', 'payed', '.'] [ True]
['Ryan', 'payed', '.'] [ True]
['M

In [27]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Intransitive_Rob_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive

In [28]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Charles'}, {'first_name': 'Kim'}, {'first_name': 'Greg'}, {'first_name': 'David'}, {'first_name': 'Louise'}, {'first_name': 'Tim'}, {'first_name': 'Julia'}, {'first_name': 'Victoria'}, {'first_name': 'Anthony'}, {'first_name': 'Edwin'}, {'first_name': 'Alice'}, {'first_name': 'Melissa'}, {'first_name': 'Donald'}, {'first_name': 'Thomas'}, {'first_name': 'Howard'}, {'first_name': 'Michael'}, {'first_name': 'Jack'}, {'first_name': 'Rachel'}, {'first_name': 'Jeff'}, {'first_name': 'Paul'}, {'first_name': 'Patrick'}, {'first_name': 'Samuel'}, {'first_name': 'Ed'}, {'first_name': 'Joseph'}, {'first_name': 'Annie'}, {'first_name': 'Claire'}, {'first_name': 'Don'}, {'first_name': 'Melissa'}, {'first_name': 'Ron'}, {'first_name': 'Kim'}, {'first_name': 'Elaine'}, {'first_name': 'Jane'}, {'first_name': 'Carolyn'}, {'first_name': 'Hugh'}, {'first_name': 'Leslie'}, {'first_name': 'Ann'}, {'first_name': 'Tim'}, {'first_name': 'Kathryn'}, {'first_name': 'Ashley'}, {'first_name

In [29]:
with open('Dataset_BERT/Transitive_Rob_BERT.txt', 'w') as f:
    print(t.data, file = f)

In [30]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Transitive_Rob_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Charles', 'payed', 'for', 'coffee', '.'] [ True]
['Kim', 'payed', 'for', 'coffee', '.'] [ True]
['Greg', 'payed', 'for', 'coffee', '.'] [ True]
['David', 'payed', 'for', 'coffee', '.'] [ True]
['Louise', 'payed', 'for', 'coffee', '.'] [ True]
['Tim', 'payed', 'for', 'coffee', '.'] [ True]
['Julia', 'payed', 'for', 'coffee', '.'] [ True]
['Victoria', 'payed', 'for', 'coffee', '.'] [ True]
['Anthony', 'payed', 'for', 'coffee', '.'] [ True]
['Edwin', 'payed', 'for', 'coffee', '.'] [ True]
['Alice', 'payed', 'for', 'coffee', '.'] [ True]
['Melissa', 'payed', 'for', 'coffee', '.'] [ True]
['Donald', 'payed', 'for', 'coffee', '.'] [ True]
['Thomas', 'payed', 'for', 'coffee', '.'] [ True]
['Howard', 'payed', 'for', 'coffee', '.'] [ True]
['Michael', 'payed', 'for', 'coffee', '.'] [ True]
['Jack', 'payed', 'for', 'coffee', '.'] [ True]
['Rachel', 'payed', 'for', 'coffee', '.'] [ True]
['Jeff', 'payed', 'for', 'coffee', 

['Tim', 'payed', 'for', 'coffee', '.'] [ True]
['Roger', 'payed', 'for', 'coffee', '.'] [ True]
['Dave', 'payed', 'for', 'coffee', '.'] [ True]
['Kevin', 'payed', 'for', 'coffee', '.'] [ True]
['Chris', 'payed', 'for', 'coffee', '.'] [ True]
['Sara', 'payed', 'for', 'coffee', '.'] [ True]
['Eric', 'payed', 'for', 'coffee', '.'] [ True]
['Dan', 'payed', 'for', 'coffee', '.'] [ True]
['Marie', 'payed', 'for', 'coffee', '.'] [ True]
['Philip', 'payed', 'for', 'coffee', '.'] [ True]
['Philip', 'payed', 'for', 'coffee', '.'] [ True]
['Alice', 'payed', 'for', 'coffee', '.'] [ True]
['Deborah', 'payed', 'for', 'coffee', '.'] [ True]
['Grace', 'payed', 'for', 'coffee', '.'] [ True]
['Greg', 'payed', 'for', 'coffee', '.'] [ True]
['Alexandra', 'payed', 'for', 'coffee', '.'] [ True]
['Sally', 'payed', 'for', 'coffee', '.'] [ True]
['Kathryn', 'payed', 'for', 'coffee', '.'] [ True]
['Adam', 'payed', 'for', 'coffee', '.'] [ True]
['Matt', 'payed', 'for', 'coffee', '.'] [ True]
['Frank', 'payed', '

In [31]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Transitive_Rob_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 